In [1]:
from tripAdvisorScraper import TripAdvisorRestaurantsScraper, TripAdvisorSpecificRestaurantScraper
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os


data_folder = './data'
data_restaurants = os.path.join(data_folder, 'restaurants.csv')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

In [2]:
button = widgets.Button(description="Download data!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print('Generating data...')
        restaurants_url = "/FindRestaurants?geo=187265&offset=0&establishmentTypes=10591&minimumTravelerRating=TRAVELER_RATING_LOW&broadened=false"
        button.layout.display = 'none'
        try: 
            restaurants_scraper = TripAdvisorRestaurantsScraper()
            restaurants_scraper.fetch_page(restaurants_url)
            corpus = restaurants_scraper.get_all_pages()
            df = pd.DataFrame(corpus)
            display(df.head())
            df.to_csv('./data/restaurants.csv', index=False)
            print('Restaurants data downloaded! run again this cell to start.')
        except Exception as e:
            print(f'Error: {e}')
            button.layout.display = 'block'

button.on_click(on_button_clicked)
if not os.path.exists(data_restaurants):
    button.layout.display = 'block'
    # with output:
        # print("Data file does not exist.")
else:
    button.layout.display = 'none'
    df = pd.read_csv(data_restaurants)
    print('Data file exists.')
    print('Select a restaurant to download its data.')

    restaurants_dropdown = widgets.Dropdown(
        options=df['restaurant_name'].to_list(),
        value=df['restaurant_name'].to_list()[0],
        description='Restaurant:',
        disabled=False,
        )
    with output:
        display(restaurants_dropdown)
        selection_button = widgets.Button(description="Get Selection")
        output_selection = widgets.Output()

        def on_selection_button_clicked(b):
            with output_selection:
                print('Generating data...')
                restaurant_name = restaurants_dropdown.value
                df_restaurants = pd.read_csv(data_restaurants)
                restaurant_data = df_restaurants[df_restaurants['restaurant_name'] == restaurant_name]
                restaurant_url = restaurant_data['restaurant_url'].values[0]
                scraper = TripAdvisorSpecificRestaurantScraper()
                scraper.fetch_page(restaurant_url)
                corpus = scraper.get_all_pages()
                df_restaurant = pd.DataFrame(corpus)
                restaurant_name = ''.join(e for e in restaurant_name if e.isalnum() or e == ' ')
                restaurant_name = restaurant_name.replace(' ', '_')
                data_restaurant = os.path.join(data_folder, f'{restaurant_name}.csv')
                df_restaurant.to_csv(data_restaurant, index=False)
                print('Selection data generated!')

        selection_button.on_click(on_selection_button_clicked)

        display(selection_button, output_selection)
        
        
        
        

Button(description='Download data!', style=ButtonStyle())

Output()

Output()

Data file exists.
Select a restaurant to download its data.
